# PTO Agent with AWS Strands SDK

This notebook demonstrates a PTO (Paid Time Off) agent built using the AWS Strands SDK and Amazon Bedrock

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from strands import Agent, tool
from strands.models.bedrock import BedrockModel

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully")


In [ ]:
# PTO data
PTO_HOURS_AVAILABLE = 94

print(f"Current PTO Balance: {PTO_HOURS_AVAILABLE} hours")


In [ ]:
# Define tools using if-else logic with the @tool decorator
@tool
def get_available_pto_hours() -> str:
    """Get the total number of PTO hours available."""
    return f"You have {PTO_HOURS_AVAILABLE} PTO hours available."

@tool
def get_available_pto_days() -> str:
    """Convert PTO hours to days (assuming 8-hour workday)."""
    days = PTO_HOURS_AVAILABLE / 8
    return f"You have {days} PTO days available ({PTO_HOURS_AVAILABLE} hours)."

@tool
def can_take_full_day() -> str:
    """Check if employee has enough PTO for a full day off (8 hours)."""
    if PTO_HOURS_AVAILABLE >= 8:
        return f"Yes, you can take a full day off. You have {PTO_HOURS_AVAILABLE} hours available."
    else:
        return f"No, you don't have enough PTO for a full day. You only have {PTO_HOURS_AVAILABLE} hours available."

@tool
def can_take_half_day() -> str:
    """Check if employee has enough PTO for a half day off (4 hours)."""
    if PTO_HOURS_AVAILABLE >= 4:
        return f"Yes, you can take a half day off. You have {PTO_HOURS_AVAILABLE} hours available."
    else:
        return f"No, you don't have enough PTO for a half day. You only have {PTO_HOURS_AVAILABLE} hours available."

print("✓ PTO tools defined with if-else logic")


## Verify AWS Credentials

Before creating the model, let's verify your AWS credentials are valid:


In [ ]:
# Check AWS credentials
import boto3
from datetime import datetime

try:
    # Create a Bedrock client to test credentials
    bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-2')
    
    # Try to get caller identity to verify credentials
    sts = boto3.client('sts', region_name='us-west-2')
    identity = sts.get_caller_identity()
    
    print("✓ AWS Credentials are valid!")
    print(f"  Account: {identity['Account']}")
    print(f"  User ARN: {identity['Arn']}")
    # print(f"  region: {identity['Region_Name']}")
    
    # If using temporary credentials, check expiration
    session = boto3.Session()
    credentials = session.get_credentials()
    if credentials.token:  # Temporary credentials have a token
        print("\n⚠️  You are using temporary credentials.")
        print("   If they expire, you'll need to refresh them.")
    
except Exception as e:
    print(f"❌ AWS Credentials Error: {e}")
    print("\n🔧 To fix this:")
    print("   1. Run: aws configure")
    print("   2. Or update your .env file with new AWS credentials")
    print("   3. Or get fresh credentials from AWS SSO/Academy")
    print("\n   Then restart the kernel and run from the beginning.")


In [ ]:
# List all available Bedrock models in your region
import boto3
import json

try:
    bedrock = boto3.client('bedrock', region_name='us-west-2')
    response = bedrock.list_foundation_models()
    
    print("🔍 Available Claude models in us-west-2:\n")
    claude_models = [
        model for model in response['modelSummaries'] 
        if 'claude' in model['modelId'].lower()
    ]
    
    for model in claude_models:
        print(f"✓ {model['modelId']}")
        if 'sonnet-4' in model['modelId']:
            print(f"  ⭐ Found Claude Sonnet 4!")
    
    if not any('sonnet-4' in m['modelId'] for m in claude_models):
        print("\n⚠️  Claude Sonnet 4 not found in us-west-2")
        print("   Recommended: Use Claude 3.5 Sonnet instead")
        
except Exception as e:
    print(f"Error listing models: {e}")


In [ ]:
# Initialize the Amazon Bedrock model
# Note: Requires AWS credentials configured via AWS CLI or environment variables
# 
# ⚠️ IMPORTANT: Claude Sonnet 4 requires an INFERENCE PROFILE, not a direct model ID!
# 
# Inference Profiles (use these for Claude Sonnet 4):
# - "us.anthropic.claude-sonnet-4-v1:0" (Claude Sonnet 4 - Cross-region, RECOMMENDED)
# - "us.anthropic.claude-3-7-sonnet-20250219-v1:0" (Claude 3.7 Sonnet)
# - "us.anthropic.claude-3-5-sonnet-20241022-v2:0" (Claude 3.5 Sonnet)
#
# Direct Model IDs (only for older models):
# - "anthropic.claude-3-haiku-20240307-v1:0" (Claude 3 Haiku)

try:
    # Using Claude 3.5 Sonnet V2 - widely available and works reliably
    model = BedrockModel(
        model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
        boto_session=session
    )
    print("✓ Bedrock model initialized successfully!")
    print("  Model: Claude 3.5 Sonnet V2")
    print("  Model ID: us.anthropic.claude-3-5-sonnet-20241022-v2:0")
    print("  Region: us-west-2")
except Exception as e:
    print(f"⚠️  Error initializing Bedrock model: {e}")
    print("\n🔧 Troubleshooting:")
    print("1. Check AWS credentials are valid (run cell 5)")
    print("2. Enable Claude Sonnet 4 in Bedrock console → Model access")
    print("3. Use inference profile (us.anthropic.claude-sonnet-4-v1:0)")
    print("4. Try different region if needed (us-east-1, us-west-2)")


In [ ]:
# Define the system prompt for the PTO agent
system_prompt = """You are a helpful HR assistant that helps employees check their PTO (Paid Time Off) hours. 
Be friendly, concise, and professional. Use the available tools to provide accurate information about PTO balances."""

# Create the PTO agent with tools
pto_agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=[
        get_available_pto_hours,
        get_available_pto_days,
        can_take_full_day,
        can_take_half_day
    ]
)

print("✓ PTO Agent created successfully with 4 tools")


In [ ]:
# TEST 1: Check available PTO hours
query = "How many PTO hours do I have available?"
print(f"User: {query}\n")

response = pto_agent(query)
print(f"Agent: {response}\n")
print("-" * 60)


## 🌐 Gradio Web Interface

Let's create an interactive web interface for the PTO agent using Gradio:


In [ ]:
# Import Gradio
import gradio as gr

def pto_agent_chat(user_query):
    """
    Wrapper function for the PTO agent to work with Gradio
    Takes a user query and returns the agent's response
    """
    try:
        response = pto_agent(user_query)
        return str(response)
    except Exception as e:
        return f"❌ Error: {str(e)}\n\nPlease make sure the agent is initialized properly."

print("✓ Gradio wrapper function created")


In [ ]:
# Create Gradio interface for PTO Agent
with gr.Blocks(title="PTO Agent - AWS Strands", theme=gr.themes.Soft()) as pto_demo:
    gr.Markdown("# 🏖️ PTO Agent (AWS Strands SDK)")
    gr.Markdown("### Powered by Claude 3.5 Sonnet V2 on Amazon Bedrock")
    
    # Display current PTO balance
    gr.Markdown(f"""
    **Current PTO Balance: {PTO_HOURS_AVAILABLE} hours ({PTO_HOURS_AVAILABLE/8:.1f} days)**
    
    Ask me anything about your PTO! I use if-else logic with AI to help you:
    - Check your available hours and days
    - See if you can take time off
    - Plan your vacation time
    """)
    
    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(
                label="💬 Ask about your PTO", 
                lines=3,
                placeholder="e.g., How many PTO hours do I have?"
            )
            submit_btn = gr.Button("🚀 Ask Agent", variant="primary", size="lg")
        
    with gr.Row():
        agent_output = gr.Textbox(
            label="🤖 Agent Response", 
            lines=10,
            show_copy_button=True
        )
    
    # Example questions
    gr.Examples(
        examples=[
            "How many PTO hours do I have available?",
            "How many days of PTO can I take?",
            "Can I take a full day off next week?",
            "Do I have enough PTO for a half day?",
            "What's my PTO balance in days?",
            "Can I take 2 weeks off?"
        ],
        inputs=user_input,
        label="📝 Example Questions (click to try)"
    )
    
    # Information section
    with gr.Accordion("ℹ️ How This Works", open=False):
        gr.Markdown("""
        **Technology Stack:**
        - 🧠 **AI Model:** Claude 3.5 Sonnet V2 (Amazon Bedrock)
        - ⚙️ **Framework:** AWS Strands SDK
        - 🛠️ **Logic:** If-else statements in Python tools
        - 🎨 **Interface:** Gradio
        
        **Available Tools:**
        1. `get_available_pto_hours()` - Returns total PTO hours
        2. `get_available_pto_days()` - Converts hours to days
        3. `can_take_full_day()` - Checks if ≥8 hours available
        4. `can_take_half_day()` - Checks if ≥4 hours available
        
        The agent automatically selects the right tool based on your question!
        """)
    
    submit_btn.click(
        fn=pto_agent_chat,
        inputs=user_input,
        outputs=agent_output
    )
    
    # Also submit on Enter key
    user_input.submit(
        fn=pto_agent_chat,
        inputs=user_input,
        outputs=agent_output
    )

print("✓ Gradio interface created")



pto_demo.launch(share=True)

In [ ]:
# Close the Gradio interface when done
pto_demo.close()


## How AWS Strands SDK Works

This agent uses AWS Strands SDK with Amazon Bedrock instead of OpenAI. Here are the key differences:

### 1. **Tool Definition with `@tool` Decorator**
- Strands uses the `@tool` decorator to define agent tools
- Each tool is a simple Python function with docstrings describing its purpose
- The if-else logic is embedded directly in each tool function

### 2. **Model Provider: Amazon Bedrock with Claude Sonnet 4**
- Uses `BedrockModel` instead of OpenAI's API
- Powered by **Claude Sonnet 4** (Anthropic's latest model) via Bedrock
- Requires AWS credentials and Bedrock model access
- Claude Sonnet 4 offers enhanced coding capabilities and reasoning

### 3. **Simpler Agent Creation**
- Create an `Agent` with:
  - `model`: The BedrockModel instance (Claude Sonnet 4)
  - `system_prompt`: Instructions for the agent's behavior
  - `tools`: List of tool functions

### 4. **Easier Invocation**
- Just call the agent like a function: `agent(query)`
- Strands handles the tool calling and response formatting automatically

### If-Else Logic Implementation:
- `get_available_pto_hours()`: Returns current hours
- `get_available_pto_days()`: Converts hours to days
- `can_take_full_day()`: Uses if-else to check if >= 8 hours
- `can_take_half_day()`: Uses if-else to check if >= 4 hours

The Strands SDK with Claude Sonnet 4 simplifies agent development with its declarative approach and seamless AWS integration!
